This notebook demonstrates strain mapping using both of the methods implemented in pyXem.

## Requirements

pyXem with https://github.com/pyxem/pyxem/pull/336 merged. Beyond Feb 1st 2019 should work

# Contents

4. <a href='#loa'> Simulate Strain Test Data</a>
5. <a href='#str'> Strain Mapping as Image Affine Transform</a>
6. <a href='#str'> Strain Mapping as Image Affine Transform</a>

Import pyXem and other required libraries

In [ ]:
%matplotlib tk
import pyxem as pxm
import numpy as np
import hyperspy.api as hs
import diffpy.structure
from matplotlib import pyplot as plt
from pyxem.generators.indexation_generator import IndexationGenerator

## <a id='loa'></a> 4. Simulate Strain Test Data

If you have data ready to go you can skip this section. We start by defining a crystal structure

In [ ]:
#An atomic structure represented using diffpy
latt = diffpy.structure.lattice.Lattice(3,3,5,90,90,90)
atom = diffpy.structure.atom.Atom(atype='Ni',xyz=[0,0,0],lattice=latt)
hexagonal_structure = diffpy.structure.Structure(atoms=[atom],lattice=latt)

We now perform our (virtual) diffraction experiment to create a single pattern

In [ ]:
ediff = pxm.DiffractionGenerator(300., 0.025)
diffraction = ediff.calculate_ed_data(hexagonal_structure, reciprocal_radius=5.)
pattern = diffraction.as_signal(128,0.05,1).data

In [ ]:
plt.imshow(pattern)

and a deformed pattern, so we can see the effect of a real space change

In [ ]:
latt = diffpy.structure.lattice.Lattice(3+0.12,3+0.12,3,90,90,90)
atom = diffpy.structure.atom.Atom(atype='Ni',xyz=[0,0,0],lattice=latt)
hexagonal_structure_d = diffpy.structure.Structure(atoms=[atom],lattice=latt)
diffractiond = ediff.calculate_ed_data(hexagonal_structure_d, reciprocal_radius=5.)
patternd = diffractiond.as_signal(128,0.05,1).data

Four copies of this pattern are made, and strained by different amounts (note we conduct the deformation in reciprocal space here to keep the code volume down)

In [ ]:
dp = pxm.ElectronDiffraction((np.asarray([[pattern,patternd],[pattern,pattern]])))

x_l = []
for x in [0, 0, -0.01, 0.02]:
    x_s = np.eye(3)
    x_s[0,0] += x
    x_l.append(x_s)

angles = hs.signals.Signal2D(np.asarray(x_l).reshape(2,2,3,3))
dp = dp.apply_affine_transformation(D=angles,order=5,inplace=False)
dp.set_diffraction_calibration(1)

In [ ]:
%matplotlib tk
dp.plot(cmap='magma')

And this data is ready to be worked on!

## <a id='loa'></a> 5. Perform Strain Mapping with an affine transform method

Create a model to the data which comprises a distorted version of a reference (unstrained) diffraction pattern at each probe position.

In [ ]:
from pyxem.components.scalable_reference_pattern import ScalableReferencePattern

In [ ]:
m = dp.create_model()
ref = ScalableReferencePattern(dp.inav[0,0])
m.append(ref)

Print the affine transform values associated with the distorted diffraction pattern before fitting.

In [ ]:
m.print_current_values()

Perform fitting

In [ ]:
m.multifit()

Construct the displacement graident tensor at each pixel from the fitting results

In [ ]:
disp_grad = ref.construct_displacement_gradient()

Perform (right) polar decomposition on the displacement gradient tensor to get rotation matrix, R, and strain matrix, U, at each pixel.

In [ ]:
R, U = disp_grad.polar_decomposition()

Get a strain map and plot it.

In [ ]:
strain_map = disp_grad.get_strain_maps()
strain_map.plot(cmap='PiYG',vmax=0.04,vmin=-0.04)

## <a id='loa'></a> 6. Perform Strain Mapping with a spot tracking method

In [ ]:
from pyxem.generators.subpixelrefinement_generator import SubpixelrefinementGenerator
from pyxem.signals.tensor_field import *
from pyxem.generators.displacement_gradient_tensor_generator import *

We start by finding the two peaks, this can be done with other peak finding methods, or by inspection (as here), although we must calibrate the signal for this

In [ ]:
dp.plot()

#help(get_DisplacementGradientMap) #this is sometimes useful to read for conventions

x_peak = [24,0]
y_peak = [0,-42]

We need to find these peaks (in each pattern) at subpixel precision

In [ ]:
spg = SubpixelrefinementGenerator(dp,np.asarray([x_peak,y_peak]))
Vs = spg.center_of_mass_method(6)

And with these objects (deformed and undeformed peaks) we are off to the races

In [ ]:
D = get_DisplacementGradientMap(hs.signals.Signal2D(Vs), Vs[0,0])

In [ ]:
strain_map = D.get_strain_maps()

In [ ]:
strain_map.plot(cmap='seismic',vmax=0.04,vmin=-0.04)